<h1>Plotly Graphing and Mapping results for Cimulation March 6</h1>

In [562]:
import pandas as pd
import geopandas as gpd

import json

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import folium
import branca
import ipywidgets as widgets
from ipywidgets import HBox, VBox

#Postgres
import psycopg2  


In [563]:
  

def setup_simulation(b):
    simulation_path = data['cim_folder']+folder_textbox.value +"/"
    agents_0_path = simulation_path+'agents_0.xlsx' # I need to grab a referene from the dictionary file!!!!
    agents_0 = pd.read_excel(agents_0_path)
    
    #print("ready for Iteration: " + str(int(itr_slider.value)))
    # + str(int(itr_slider.value))



dMenu = widgets.Dropdown(
    options=['Population Change', '2', '3'],
    value='Population Change',
    description='Select Statistics:',
    disabled=True,
    )

setup_btn = widgets.ToggleButton(
    value=False,
    description='Setup',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
itr_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=18,
    step=1,
    description='Iteration:',
    readout=True,
    readout_format='d',
    )

button = widgets.Button(description="Setup")
button.on_click(setup_simulation)

folder_textbox = widgets.Text(
    value='agents_cim_0317_1615',
    description='Folder:',
    )

simulation_textbox = widgets.Text(
    value='cim_0315_1240',
    description='Folder:',
    )

tab1 = VBox(children=[folder_textbox,
                      itr_slider,
                      button
                      ])

tab2 = VBox(children=[dMenu,
                      ])
tab = widgets.Tab(children=[tab1, tab2])
tab.set_title(0, 'Iteration Setup')
tab.set_title(1, 'statistics test')




In [564]:
def iteration_fig(dti_itr, op=1):

    fig = make_subplots(rows=2, cols=1)


    #fig.update_layout(height=600, width=800, title_text="Subplots")


    
    x = np.array(np.array(dti_itr.index))
    y_Stay = np.array(np.array(dti_itr['staying']))
    y_Leave= np.array(np.array(dti_itr['laeving']))
    y_Come= np.array(np.array(dti_itr['new comers']))
    y_PopSize= np.array(np.array(dti_itr['Pop Size']))
    y_Native = np.array(np.array(dti_itr['Native Age']))
    
    fig.add_trace(go.Scatter(x=x, y=y_PopSize, name="Pop Size",line_shape='spline',
                             line = dict(color='firebrick', width=4, dash='dash')),row=1, col=1)
    
    fig.add_trace(go.Scatter(x=x, y=y_Native, name="Native  Age",line_shape='spline',
                             line = dict(color='black', width=2, dash='dot')),row=1, col=1)
    
    fig.add_trace(go.Scatter(x=x, y=y_Stay, name="Staying",line_shape='spline'),row=1, col=1)

    fig.add_trace(go.Scatter(x=x, y=y_Leave , name="Leaving",hoverinfo='text+name',line_shape='spline'),row=1, col=1)

    fig.add_trace(go.Scatter(x=x, y=y_Come , name="New Comers",hoverinfo='text+name',line_shape='spline'),row=1, col=1)

    #-------------------------------------------subplot-----------------------
    y_High = np.array(np.array(dti_itr['High Income']))
    y_Medium = np.array(np.array(dti_itr['Medium Income']))   
    y_Low= np.array(np.array(dti_itr['Low Income']))

    fig.add_trace(go.Scatter(x=x, y=y_High, name="High Income",hoverinfo='text+name',line_shape='spline'),row=2, col=1)

    fig.add_trace(go.Scatter(x=x, y=y_Medium , name="Medium Income",hoverinfo='text+name',line_shape='spline'),row=2, col=1)

    fig.add_trace(go.Scatter(x=x, y=y_Low , name="Low Income",hoverinfo='text+name',line_shape='spline'),row=2, col=1)


    
    fig.update_traces(hoverinfo='text+name', mode='lines+markers')
    fig.update_layout(height=800, width=800,legend=dict(y=0.5, traceorder='reversed', font_size=16))
    
    

    return fig

def prepare_indicators(dt,i,j):
    
    fig = go.Figure()
    #-------------------Ownership-------------------
    c = dt.loc[j]['owner']
    p = dt.loc[i]['owner']
    fig.add_trace(go.Indicator(
        mode = "number+delta",
        value = (c),
        title = {"text": "<span style='font-size:0.8em;color:Green'>Ownership</span>"},
        domain = {'x': [0.0, 0.25], 'y': [0.8, 0.95]},
        delta = {'reference': (p), 'relative': True, 'position' : "top"}))

    c = dt.loc[j]['rent']
    p = dt.loc[i]['rent']
    
    fig.add_trace(go.Indicator(
        mode = "number+delta",
        value = (c),
        title = {"text": "<span style='font-size:0.8em;color:limegreen'>Rent</span>"},
        domain = {'x': [0.0, 0.25], 'y': [0.55, 0.7]},
        delta = {'reference': (p), 'relative': True, 'position' : "top"}))

    #-------------------Income-------------------
    c = dt.loc[j]['High Income']
    p = dt.loc[i]['High Income']
    fig.add_trace(go.Indicator(
        mode = "number+delta",
        value = (c),
        title = {"text": "<span style='font-size:0.8em;color:Green'>High Income</span>"},
        domain = {'x': [0.3, 0.45], 'y': [0.8, 0.95]},
        delta = {'reference': (p), 'relative': True, 'position' : "top"}))
        
    c = dt.loc[j]['Medium Income']
    p = dt.loc[i]['Medium Income']
    fig.add_trace(go.Indicator(
        mode = "number+delta",
        value = (c),
        title = {"text": "<span style='font-size:0.8em;color:Orange'>Medium Income</span>"},
        domain = {'x': [0.3, 0.45], 'y': [0.55, 0.7]},
        delta = {'reference': (p), 'relative': True, 'position' : "top"}))
   
    c = dt.loc[j]['Low Income']
    p = dt.loc[i]['Low Income']    
    fig.add_trace(go.Indicator(
        mode = "number+delta",
        value = (c),
        title = {"text": "<span style='font-size:0.8em;color:Red'>Low Income</span>"},
        domain = {'x': [0.3, 0.45], 'y': [0.35, 0.49]},
        delta = {'reference': (p), 'relative': True, 'position' : "top"}))

    c = dt.loc[j]['Average Income']
    p = dt.loc[i]['Average Income']     
    fig.add_trace(go.Indicator(
        mode = "number+delta",
        value = (c),
        title = {"text": "Average Income"},
        domain = {'x': [0.3, 0.45], 'y': [0.05, 0.25]},
        delta = {'reference': (p), 'relative': True, 'position' : "top"}))
    #-------------------Income-------------------


    #-------------------Age-------------------
    c = dt.loc[j]['Old']
    p = dt.loc[i]['Old']   
    fig.add_trace(go.Indicator(
        mode = "number+delta",
        value = (c),
        title = {"text": "Old"},
        domain = {'x': [0.45, 0.7], 'y': [0.8, 0.95]},
        delta = {'reference': (p), 'relative': True, 'position' : "top"}))

    c = dt.loc[j]['Young']
    p = dt.loc[i]['Young']   
    fig.add_trace(go.Indicator(
        mode = "number+delta",
        value = (c),
        title = {"text": "Young"},
        domain = {'x': [0.45, 0.7], 'y': [0.55, 0.7]},
        delta = {'reference': (p), 'relative': True, 'position' : "top"}))

    c = dt.loc[j]['Average Age']
    p = dt.loc[i]['Average Age']   
    fig.add_trace(go.Indicator(
        mode = "number+delta",
        value = (c),
        title = {"text": "Average Age"},
        domain = {'x': [0.45, 0.7], 'y': [0.05, 0.25]},
        delta = {'reference': (p), 'relative': True, 'position' : "top"}))

    #-------------------Income-------------------

#Native Age
    
    c = dt.loc[j]['Pop Size']
    p = dt.loc[i]['Pop Size']      
    fig.add_trace(go.Indicator(
        mode = "number+delta",
        value = (c),
        title = {"text": "<span style='font-size:0.8em;color:black'><b>Population <br> present:</b></span>"},
        delta = {'reference': (p), 'relative': True},
        domain = {'x': [0.0, 0.3], 'y': [0.05, 0.25]}))

    c = dt.loc[j]['Native Age']
    p = dt.loc[i]['Native Age'] 
    fig.add_trace(go.Indicator(
        mode = "number+delta",
        value = (c),
        title = {"text":"<span style='font-size:0.8em;color:gray'>Average <br> Neighborhood<br>Seniority</span>" },
        delta = {'reference': (p), 'relative': True},
        domain = {'x': [0.7, 0.9], 'y': [0.05, 0.25]}))
    #"Accounts<br><span style='font-size:0.8em;color:gray'>Subtitle</span><br><span style='font-size:0.8em;color:gray'>Subsubtitle</span>"
#    


    fig.update_layout(height=500, width=800)
    return fig
    #setup = 2
    

In [565]:
def Convert_2039_2_4326(feature):
    #Convert From ESPG 2039 (Israel TM Grid) to ESPG 4326 (WGS 84)
    try:
        #step 1 
        interFeature = feature.to_crs("+proj=tmerc +lat_0=31.7343936111111 +lon_0=35.2045169444445 +k=1.0000067 +x_0=219529.584 +y_0=626907.39 +ellps=GRS80 +towgs84=-23.772,-17.490,-17.859,-0.31320,-1.85274,1.67299,5.4262 +units=m +no_defs")
        crs_4326 =  {'init': 'epsg:4326'}
        #step 2
        Feature_4326 = interFeature.to_crs(crs_4326)
        return (Feature_4326)

    except Exception as e:
        print(e)
    else:
        print("Conversion to WGS 84 Successful")

def Convert_4326_2_2039(feature):
    #Convert From ESPG 4326 (WGS 84) To ESPG 2039 (Israel TM Grid)
    try:
        #step 1 
        interFeature = feature.to_crs("+proj=tmerc +lat_0=31.7343936111111 +lon_0=35.2045169444445 +k=1.0000067 +x_0=219529.584 +y_0=626907.39 +ellps=GRS80 +towgs84=-23.772,-17.490,-17.859,-0.31320,-1.85274,1.67299,5.4262 +units=m +no_defs")
        crs_2039 =  {'init': 'epsg:2039'}
        #step 2
        Feature_2039 = interFeature.to_crs(crs_2039)
        return (Feature_2039)
        
    except Exception as e:
        print(e)
    else:
        print("Conversion to Israel TM Grid Successful")
            
def get_itr_row(ag, preSetup=False):

    ag['ones'] = 1
    #filter only the people who are staying
    
    ag_1 = ag[ag['Agent_status']!='leaving']
    pop = ag_1.AgentID.count() #number of population
    
    #agents_i[agents_i['Agent_status']!='leaving'].AgentID.count()
    
    pivot_ownership = ag_1.pivot(index='AgentID',columns='OwnershipDesc',values='ones')
    del pivot_ownership.index.name
    p_ownership = pivot_ownership.count() #ownership
    pd_ownership = pd.DataFrame(p_ownership).transpose()
    del pd_ownership.columns.name
    
    pivot_income = ag_1.pivot(index='AgentID',columns='IncomeDesc',values='ones')
    del pivot_income.index.name
    p_income = pivot_income.count() #count income by number
    pd_income    = pd.DataFrame(p_income).transpose()
    del pd_income.columns.name
    
    populationAverageIncome = int(ag_1.AgentIncome.mean()) #Average Income
    
    Pivot_Age = ag_1.pivot(index='AgentID',columns='AgeGroupDesc',values='ones')
    del Pivot_Age.index.name
    P_age = Pivot_Age.count() #pivot Age
    populationAverageAge = int(ag_1.Age.mean()) #average Age
    pd_age       = pd.DataFrame(P_age).transpose()
    del pd_age.columns.name
    
    nativeAverage = ag_1.Native_Seniority.mean() #average Nativity
    
        
        
    df = pd.DataFrame()
    df[['owner','rent']] = pd_ownership
    df[['High Income','Low Income','Medium Income']] = pd_income
    df['Average Income'] = populationAverageIncome
    df[['Old','Young']] = pd_age
    df['Average Age'] = populationAverageAge
    df['Native Age'] = nativeAverage
    df['Pop Size'] = pop
    
    if (preSetup == False):

        p_move =  ag[['laeving','new comers','staying']].count()
        pd_move = pd.DataFrame(p_move).transpose()
        del pd_move.columns.name
        df[['laeving','new comers','staying']]=pd_move
    
    return df


<h2>Step 1: Loading Dicitonary Files</h2>


## Reference to the dictionary File
### Dictionary File is important to keep a reference of the main files for the simulation

In [566]:
#reference dictionary file
path_1 = 'library.json'
with open(path_1) as json_file:
    data = json.load(json_file)

json_formatted_str = json.dumps(data, indent=2)
print(json_formatted_str)


{
  "GIS_package": {
    "old_bldgs": {
      "package": "Source/data/GIS/BaseLayers.gpkg",
      "layer": "bldgs_renewal_2020",
      "database": "base_layers",
      "feature": "bldgs_renewal_2020"
    },
    "old_bldgs_background": {
      "package": "Source/data/GIS/BaseLayers.gpkg",
      "layer": "bldgs_nonrenwal2020",
      "database": "base_layers",
      "feature": "bldgs_nonrenewal_2020"
    },
    "updated_bldg": {
      "package": "Source/data/GIS/sim_updated.gpkg",
      "layer": "sim_march8_current_bldgs",
      "database": "sim_updated",
      "feature": "accumulated"
    }
  },
  "script_file": "data/script/script_ssi.xlsx",
  "script_file_old": "Source/data/script/script_ssi_march_8_535pm.xlsx",
  "future_script": "data/FutureBldgs_Tama_38.xlsx",
  "Buildings_ref": "data/Building_flat.xlsx",
  "Future_ref": "data/FutureBldgs_All.xlsx",
  "bldg_path": "data/bldg/",
  "Agents": "Source/Agents_8pm.xlsx",
  "cim_folder": "simulationID/",
  "Google Drive Prepath": "content/

In [567]:
wpass = widgets.Password(
    value='password',
    placeholder='Enter password',
    description='Password:',
    disabled=False
)
display(wpass)

Password(description='Password:', placeholder='Enter password')

In [569]:
#Connection to Postgres Server
con = psycopg2.connect(database="bat_yam_simulation_march2020", user="Shai",password=wpass.value, host="localhost")



In [570]:
#Setting up geographic files

In [571]:
simulation_folder = widgets.Text(
    value='cim_0315_1240',
    description='Folder:'
    )
display(simulation_folder)

Text(value='cim_0315_1240', description='Folder:')

In [572]:
#Reference to the database names
db_old_bldg =  data["GIS_package"]["old_bldgs"]["database"]
feature_old_bldg = data["GIS_package"]["old_bldgs"]["feature"]

db_bg_old_bg_bldg =  data["GIS_package"]["old_bldgs_background"]["database"]
feature_old_bg_bldg = data["GIS_package"]["old_bldgs_background"]["feature"]

#SQL commands
sql_old_bldg = 'SELECT * FROM '+db_old_bldg+'.'+feature_old_bldg
sql_old_bg_bldg = 'SELECT * FROM '+db_bg_old_bg_bldg+'.'+feature_old_bg_bldg

#Location of the simulation folder
sim_folder='simulationID/'+simulation_folder.value+'/'
#'simulationID/cim_0315_1240/
script_path = sim_folder+data["script_file"]

#getting old bldgs
old_bldgs = gpd.GeoDataFrame.from_postgis(sql_old_bldg, con, geom_col='geom' )
old_all_bldgs = gpd.GeoDataFrame.from_postgis(sql_old_bg_bldg, con, geom_col='geom' )

In [573]:
data["script_file"]

'data/script/script_ssi.xlsx'

In [574]:
#pack_old_bldg = data["GIS_package"]["old_bldgs"]["package"]
#layer_old_bldg = data["GIS_package"]["old_bldgs"]["layer"]

#pack_all_old_bldg = data["GIS_package"]["old_bldgs_background"]["package"]
#layer_all_old_bldg = data["GIS_package"]["old_bldgs_background"]["layer"]
#old_bldgs = gpd.read_file(pack_old_bldg,layer = layer_old_bldg,driver='GPKG')
#old_all_bldgs = gpd.read_file(pack_all_old_bldg,layer = layer_all_old_bldg,driver='GPKG')
script_file = pd.read_excel(sim_folder+data["script_file"])
ref_flat_file = pd.read_excel(sim_folder+data["Buildings_ref"])

future_bldgs = pd.read_excel(sim_folder+data["Future_ref"]) #Future reference buildings



In [575]:
#temporary fix - Need to add 'FutureBuilding' in the  ssi
ref_flat_file['FutureBuilding']=False

In [576]:

bldg_list = ref_flat_file[['bld_address', 'OriginalUnits', 'OriginalFloors',
       'OriginalHouseSize', 'purchase_p', 'rent_price','FutureBuilding']]

In [577]:
#bldg_list = script_file[script_file['pointPolygon']=='Point'][['bld_address', 'OriginalUnits', 'OriginalFloors',
#       'OriginalHouseSize', 'purchase_p', 'rent_price', 'Arnona', 'Old_Purchase', 'OldRent',
#       'Tax and Maintenace']]
#list_of_excels = script_file[script_file['pointPolygon']=='Polygon']['ExcelPolygon']

In [578]:
sim_folder+data["Buildings_ref"]

'simulationID/cim_0315_1240/data/Building_flat.xlsx'

In [579]:
select_bldg = old_bldgs['bld_id'].isin(ref_flat_file.bld_address)
select_bldg_ds = old_bldgs[select_bldg][['bld_id','geom']]
select_bldg_ds.reset_index(inplace=True)



In [580]:
cb_list = bldg_list[bldg_list['FutureBuilding']==False]
select_bldg_ds_join = pd.merge(select_bldg_ds,cb_list,how='left',left_on='bld_id',right_on='bld_address')
select_bldg_wgs_84 = Convert_2039_2_4326(select_bldg_ds_join)

### get center point

In [581]:
lon = select_bldg_wgs_84.geometry.centroid.x.mean()
lat = select_bldg_wgs_84.geometry.centroid.y.mean()

latlng=[lat, lon]
zoom = 16
fmap = folium.Map(latlng,zoom_start=zoom,tiles='cartodbpositron')
fmap2 = folium.Map(latlng,zoom_start=zoom,tiles='cartodbpositron')

### add background buildings

In [582]:
old_all_bldgs_wgs84 = Convert_2039_2_4326(old_all_bldgs)

In [583]:
gjson = old_all_bldgs_wgs84.to_json()
style_1 ={'fillOpacity': 0.25,'weight': 0,'fillColor': '##cc0000'}
folium.GeoJson(gjson,
               style_function = lambda x: style_1).add_to(fmap)
folium.GeoJson(gjson,
               style_function = lambda x: style_1).add_to(fmap2)

In [584]:
gjson = old_all_bldgs_wgs84.to_json()
style_1 ={'fillOpacity': 0.25,'weight': 0,'fillColor': '##cc0000'}
folium.GeoJson(gjson,
               style_function = lambda x: style_1).add_to(fmap)
folium.GeoJson(gjson,
               style_function = lambda x: style_1).add_to(fmap2)

gjson = select_bldg_wgs_84.to_json()
style_2 ={'fillOpacity': 0.75,'weight': 1,'fillColor': '#ff0000'}
folium.GeoJson(gjson,
               style_function = lambda x: style_2,
               tooltip=folium.GeoJsonTooltip(fields=['bld_address',
                                                     'OriginalUnits',
                                                     'OriginalFloors',
                                                     'OriginalHouseSize',
                                                     'purchase_p',
                                                     'rent_price'
                                                     ],
                                             aliases=['Building Address Code',
                                                     'Number of Units',
                                                     'OriginalFloors',
                                                     'Average house size',
                                                     'Price Per Meter',
                                                     'Rent Per Meter'                                                
                                                      ])
              ).add_to(fmap)
display(fmap)

## Joining the GIS and this list

<h2>Step 2: Loading Essential Files from simulation and GIS files</h2>



## Setting iteration for simulation

## Choosing Analytics

In [585]:
setup=0
j=0
VBox(children=[tab])

In [653]:
if (setup==0):
    simulation_path = sim_folder+folder_textbox.value +"/"
    agents_0_path = simulation_path+'agents_0.xlsx' # I need to grab a referene from the dictionary file!!!!
    agents_0 = pd.read_excel(agents_0_path)
    
    setup = 1
 

dti = get_itr_row(agents_0,True)
i = int(itr_slider.value)
agents_itr = "agents_itr_"+str(int(i))+".xlsx"
agents_i_path = simulation_path+agents_itr # I need to grab a referene from the dictionary file!!!!
agents_i = pd.read_excel(agents_i_path)
if (i==0) & (j==0):
    dti_itr = dti
j= i+1
next_row =  get_itr_row(agents_i)
j=i+1
dti_itr = dti_itr.append(next_row)
dti_itr = dti_itr[['Average Age', 'Average Income', 'High Income', 'Low Income',
       'Medium Income', 'Native Age', 'Old', 'Pop Size', 'Young',
       'laeving', 'new comers', 'owner', 'rent', 'staying']]
dti_itr.reset_index(inplace=True)
fig_indicators = prepare_indicators(dti_itr,i,j)
fig = iteration_fig(dti_itr)

fig_indicators.show()
fig.show()

## 1. Wendesday Need to link between New Buildings or Points to who stayed and who left
## 2. Wendesday Improve interface

In [652]:
row_1 = script_file.loc[i]
bld_address = row_1['bld_address']
bld_address_title = row_1['Address Title']
project_type = row_1['TypeTitle']
future_plan = row_1['FuturePlanID']
gis_file = row_1['GIS_files']
bld_operation = row_1['bld_operation']

fb = future_bldgs[['bld_address','TotalUnits','TotalFloors','AvrgTotaalArea','newMaintenace','Purchase','Rent','address_text']]
pack   = data["GIS_package"]["updated_bldg"]["database"]
layer1 = gis_file

#SQL commands
sql_com = 'SELECT * FROM '+pack+'.'+layer1

#getting old bldgs
specific_bldg = gpd.GeoDataFrame.from_postgis(sql_com, con, geom_col='geom' )
specific_bldg = specific_bldg[['bld_id','geom']]
if (row_1.bld_operation>2):
    lob = specific_bldg.bld_id.apply(lambda x:x.strip()).tolist()
    specific_bldg['bld_id'] = lob

if (row_1.bld_operation<4):
    bld_i = 'bldg_itr_'+str(int(i))+'.xlsx'
    bld_i_path = simulation_path+bld_i # I need to grab a referene from the dictionary file!!!!
    bld_i_for_gis = pd.read_excel(bld_i_path)
    bld_i_for_gis_data = bld_i_for_gis[['AgentBldAdd','laeving','new comers','staying','New Total Residence','Old Total Residence','Percent Left','Percent New Comers','Type of Project']]
    if (row_1.bld_operation>2):
        lob = specific_bldg.bld_id.tolist()
        specific_bldg['bld_id'] = lob
        selected_row_data = bld_i_for_gis_data[bld_i_for_gis_data.AgentBldAdd.isin(lob)]
        #specific_bldg.ffill(axis = 0) 
    else:
        selected_row_data = bld_i_for_gis_data[bld_i_for_gis_data.AgentBldAdd.isin(specific_bldg.bld_id)]
    specific_bldg = pd.merge(specific_bldg,selected_row_data, how='left', left_on='bld_id',right_on='AgentBldAdd')
    #future_bldgs
    specific_bldg = pd.merge(specific_bldg,fb, how='left', left_on='bld_id',right_on='bld_address')
    
    if (row_1.bld_operation>2):
        col = specific_bldg.columns.drop('geom')
        fna = specific_bldg[col].fillna(0)
        fna['geom'] = specific_bldg.geometry
        specific_bldg = gpd.GeoDataFrame(fna,crs = specific_bldg.crs,  geometry = 'geom')

#specific_bldg = gpd.read_file(pack, layer = layer1, driver="GPKG")
specific_bldg_wgs84 = Convert_2039_2_4326(specific_bldg)

lon = specific_bldg_wgs84.geometry.centroid.x.values[0]
lat = specific_bldg_wgs84.geometry.centroid.y.values[0]

if (int(i)==0):
    fmap2 = createMap()
    fmap3 = AddLayerChooseView(fmap2)
fmap3 = AddLayerChooseView(fmap3, layer=specific_bldg_wgs84, lat = lat, lon=lon, zoom=18, option=1)




DatabaseError: Execution failed on sql 'SELECT * FROM sim_updated.bldgs_17a': relation "sim_updated.bldgs_17a" does not exist
LINE 1: SELECT * FROM sim_updated.bldgs_17a
                      ^


In [649]:
print(' :כתובת '+ bld_address_title )
print('project type '+ project_type )


display(fmap3)

 :כתובת דוד מסיקה 6,8 והרצל 8
project type תב״ע פינוי בינוי


In [650]:
fmap3.save('map_march_18_5_26pm.html')

In [37]:
specific_bldg_wgs84.crs
select_bldg_israel_tm  = Convert_4326_2_2039(select_bldg_wgs_84)

In [38]:
select_bldg_israel_tm.crs

{'init': 'epsg:2039'}

In [39]:
scehem = data["GIS_package"]["updated_bldg"]["database"]
feat = data["GIS_package"]["updated_bldg"]["feature"]
#select_bldg_israel_tm  = Convert_4326_2_2039(select_bldg_wgs_84)
#select_bldg_israel_tm.to_file(pack, layer = layer1, driver="GPKG")

In [40]:
def createMap(layer=old_all_bldgs_wgs84, lat=32.0272641731828, lon = 34.74246651579093, zoom=16):
    

    latlng=[lat, lon]
    zoom = zoom
    fmap = folium.Map(latlng,zoom_start=zoom,tiles='cartodbpositron')
    gjson = layer.to_json()
    style_1 ={'fillOpacity': 0.25,'weight': 0,'fillColor': '##cc0000'}
    folium.GeoJson(gjson,
                   style_function = lambda x: style_1).add_to(fmap)



    return fmap

In [297]:
def AddLayerChooseView(fmap , layer=select_bldg_wgs_84, lat=32.0272641731828, lon = 34.74246651579093, zoom=20, option=0):
    

    fmap.location = [lat, lon]
    fmap.options['zoom']=20
    gjson = layer.to_json()
    if (option == 0):
        style_2 ={'fillOpacity': 0.25,'weight': 1,'fillColor': '#ff0000'}
        tooltip1=folium.GeoJsonTooltip(fields=['bld_address'],
                                                 aliases=['Building Address Code'])
        

#fb = future_bldgs[['bld_address','TotalUnits','TotalFloors','AvrgTotaalArea','newMaintenace','Purchase','Rent','address_text']]
        
    else:
        style_2 ={'fillOpacity': 0.75,'weight': 1,'fillColor': '#ff0000'}
        tooltip1=''
        tooltip1=folium.GeoJsonTooltip(fields=['bld_id',
                                               'laeving',
                                               'new comers',
                                               'staying',
                                               'New Total Residence',
                                               'Old Total Residence',
                                               'Percent Left',
                                               'Percent New Comers',
                                               'Type of Project',
                                               'TotalUnits',
                                               'TotalFloors',
                                               'AvrgTotaalArea',
                                               'newMaintenace',
                                               'Purchase',
                                               'Rent',
                                               'address_text'
    
                                              ],
                                       aliases=['Building Code:',
                                                'Leaving:',
                                                'Number of new comers:',
                                                'Number of staying:',
                                                'New Total Residence:',
                                                'Old Total Residence:',
                                                'Percent Left:',
                                                'Percent new comers:',
                                                'Type of project:',
                                                'Units:',
                                                'Floors:',
                                                'Average Area:',
                                                'Maintenance Cost:',
                                                'Purchase:',
                                                'Rent and Maintence:',
                                                'Address:'
                                               ])
         
        
    folium.GeoJson(gjson,
                   style_function = lambda x: style_2,
                   tooltip=tooltip1
                  ).add_to(fmap)

    return fmap

In [152]:
specific_bldg_wgs84.columns

Index(['id', 'geom', 'fid', 'index', 'bld_id', 'bld_address', 'OriginalUnits',
       'OriginalFloors', 'OriginalHouseSize', 'purchase_p', 'rent_price',
       'FutureBuilding', 'floors', 'height', 'AgentBldAdd', 'laeving',
       'new comers', 'staying', 'New Total Residence', 'Old Total Residence',
       'Percent Left', 'Percent New Comers', 'Type of Project'],
      dtype='object')

In [78]:
    lon = select_bldg_wgs_84.geometry.centroid.x.mean()
    lat = select_bldg_wgs_84.geometry.centroid.y.mean()

    folium.GeoJson(gjson,
                   style_function = lambda x: style_1).add_to(fmap2)
    

    gjson = select_bldg_wgs_84.to_json()
    style_2 ={'fillOpacity': 0.75,'weight': 1,'fillColor': '#ff0000'}
    folium.GeoJson(gjson,
                   style_function = lambda x: style_2,
                   tooltip=folium.GeoJsonTooltip(fields=['bld_address',
                                                         'OriginalUnits',
                                                         'OriginalFloors',
                                                         'OriginalHouseSize',
                                                         'purchase_p',
                                                         'rent_price'
                                                        ],
                                                 aliases=['Building Address Code',
                                                         'Number of Units',
                                                         'OriginalFloors',
                                                         'Average house size',
                                                         'Price Per Meter',
                                                         'Rent Per Meter'
                                                         ])
                  ).add_to(fmap)

In [48]:
lon

34.74246651579093

# functions and preprocessing

## Experimental!!



In [34]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

## Curtosy to a book I need!!!
https://ipython-books.github.io

In [32]:
widgets.Dropdown(
    options=['1', '2', '3'],
    value='2',
    description='Number:',
    disabled=False,
)

Dropdown(description='Number:', index=1, options=('1', '2', '3'), value='2')

In [33]:
freq_slider = widgets.FloatSlider(
    value=2.,
    min=1.,
    max=10.0,
    step=0.1,
    description='Frequency:',
    readout_format='.1f',
)

range_slider = widgets.FloatRangeSlider(
    value=[-1., +1.],
    min=-5., max=+5., step=0.1,
    description='xlim:',
    readout_format='.1f',
)
color_picker = widgets.ColorPicker(
    concise=True,
    description='Background color:',
    value='#efefef',
)

button = widgets.Button(
    description='Plot',
)

color_buttons = widgets.ToggleButtons(
    options=['blue', 'red', 'green'],
    description='Color:',
)
title_textbox = widgets.Text(
    value='Hello World',
    description='Title:',
)

grid_button = widgets.ToggleButton(
    value=False,
    description='Grid',
    icon='check'
)
tab1 = VBox(children=[freq_slider,
                      range_slider,
                      ])
tab2 = VBox(children=[color_buttons,
                      HBox(children=[title_textbox,
                                     color_picker,
                                     grid_button]),
                                     ])

tab = widgets.Tab(children=[tab1, tab2])
tab.set_title(0, 'plot')
tab.set_title(1, 'styling')
VBox(children=[tab, button])

NameError: name 'VBox' is not defined

In [72]:
title_textbox1 = widgets.Text(
    value='Hello World',
    description='Title:',
)
display(title_textbox1)


Text(value='Hello World', description='Title:')

In [26]:
itr_slider.value

0.0

In [27]:
@widgets.interact_manual(
    color=['blue', 'red', 'green'], lw=(1., 10.))
def plot(freq=1., color='blue', lw=2, grid=True):
    t = np.linspace(-1., +1., 1000)
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    ax.plot(t, np.sin(2 * np.pi * freq * t),
            lw=lw, color=color)
    ax.grid(grid)

interactive(children=(FloatSlider(value=1.0, description='freq', max=3.0, min=-1.0), Dropdown(description='col…

In [28]:
@widgets.interact_manual(
    color=['blue', 'red', 'green'], lw=(1., 10.))

SyntaxError: unexpected EOF while parsing (<ipython-input-28-4afd22bed952>, line 2)